In [1]:
from tkinter import *
from tkinter import filedialog
import tkinter.messagebox as box
import PIL.Image, PIL.ImageTk
import cv2
import time
import os
import numpy as np
import math
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial import distance as dist
from collections import OrderedDict

In [14]:
#Get raw video frame and frame size frome video capture
class MyVideoCapture:  #OK
    def __init__(self, video_source=0):
        # Open the video source
        self.vid = cv2.VideoCapture(video_source)
        #check rotation
        #self.rotateCode = self.check_rotation(video_source)
        
        if not self.vid.isOpened():
            raise ValueError("Unable to open video source", video_source)

        # Get video source width and height
        self.width_origin = self.width = self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.height_origin = self.height = self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
        
        #init
        self.video_frameCount = self.vid.get(cv2.CAP_PROP_FRAME_COUNT)
        self.video_farmePos = self.vid.get(cv2.CAP_PROP_POS_FRAMES )
        self.video_milisec = self.vid.get(cv2.CAP_PROP_POS_MSEC)
        #adjust video size to fit window
        self.overSize = 0
        if self.height_origin >400:
            self.size_multiplyer = 400/self.height_origin
            self.width = round(self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)*self.size_multiplyer)
            self.height = round(self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)*self.size_multiplyer)
            self.overSize = 1
    
    #seek the video to the selected frame
    def set_vid_frame(self,framePos):
        if self.vid.isOpened():
            self.vid.set(cv2.CAP_PROP_POS_FRAMES, framePos)
    
    #get frame count, frame position and video duration in millisec 
    def get_vid_detail(self):
        if self.vid.isOpened():
            
            return (self.video_frameCount,self.video_farmePos,self.video_milisec)
        else: 
            return None
    
    #read frame from the video and return image frame
    def get_frame(self):
        if self.vid.isOpened():
            
            self.video_farmePos = self.vid.get(cv2.CAP_PROP_POS_FRAMES )
            self.video_milisec = self.vid.get(cv2.CAP_PROP_POS_MSEC)
            ret, frame = self.vid.read()
            if ret:
                #frame = self.correct_rotation(frame, self.rotateCode)
                # Return a boolean success flag and the current frame converted to BGR
                return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            else:
                return (ret, None)
        else:
            return (ret, None)

    # Release the video source when the object is destroyed
    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()        
    
    #For some reason video taken by mobile phone make a video turn unsidedown
    #This method should solve that problem but still need more work on it
    def check_rotation(self,path_video_file):
        # this returns meta-data of the video file in form of a dictionary
        meta_dict = ffmpeg.probe(path_video_file)

        # from the dictionary, meta_dict['streams'][0]['tags']['rotate'] is the key
        # we are looking for
        rotateCode = None
        if int(meta_dict['streams'][0]['tags']['rotate']) == 90:
            rotateCode = cv2.ROTATE_90_CLOCKWISE
        elif int(meta_dict['streams'][0]['tags']['rotate']) == 180:
            rotateCode = cv2.ROTATE_180
        elif int(meta_dict['streams'][0]['tags']['rotate']) == 270:
            rotateCode = cv2.ROTATE_90_COUNTERCLOCKWISE

        return rotateCode
    
    def correct_rotation(self,frame, rotateCode):  
        return cv2.rotate(frame, rotateCode)

In [3]:
#convert a color image into a binary image
#get_image() return binary image
class BinaryImage:
    def __init__(self,frame,threshValue = 250):
        #convert color image to grayscale image
        self.imgray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        #blur the grayscale image
        self.imblur1 = cv2.medianBlur(self.imgray,5)
        #blur the grayscale image
        self.imblur2 = cv2.GaussianBlur(self.imblur1,(5,5),0)
        #convert the grayscale image to binary image
        ret,self.thresh = cv2.threshold(self.imblur1,threshValue,255,cv2.THRESH_BINARY)
    def get_image(self):
        return self.thresh
        
#Read all contours in a binary image and get a coordinate of those contours
#get_list() return list if contours coordinate
    #example:[(1244,486),(486,352)]
#draw() return an image of all contours centroids
class Contour:
    def __init__(self,binary_frame):
        self.binary_frame = binary_frame
        self.contours_coor = []
        self.contours, self.hierarchy = cv2.findContours(self.binary_frame,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        for c in self.contours:
            # calculate moments for each contour
            M = cv2.moments(c)
            # calculate x,y coordinate of center
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
            else:
                cX = 0
                cY = 0
            #add list of new coordinate to the previous one
            self.contours_coor.append((cX,cY))
    def get_list(self):
        return self.contours_coor
    def draw(self,frame,textSize = 0.5):
        self.frame=frame
        self.textSize = textSize
        for c in self.contours_coor:
            cv2.circle(self.frame,c, round(self.textSize*6), (0,0,255), -1)
        return self.frame

    
    
#Update labeled markers with all contours ID and markers set point
#get_list() return list of labeled marker with coordinate in that frame position
    #example
        #input
            #framePos = 32
            #markerID = [[0, 402, 318], [1, 402, 287], [2, 410, 262]]
            #markerSetPoint = [[21, [['KNE', '1'], ['THI', '2'], ['TIB', '0']]], [82, [['KNE', '1'], ['THI', '0'], ['TIB', '2']]]]
        #output
            #match = [[KNE,402,278],[THI,410,262],[TIB,402, 318]]
#draw() return an imgae of labeled markers
class MarkerUpdate:###<----------this class is not in use
    def __init__(self,markerID,markerSetPoint,framePos):  
        self.match = []
        for i in range(len(markerSetPoint)):
            if i is not len(markerSetPoint)-1:
                if framePos >= markerSetPoint[i][0] and framePos < markerSetPoint[i+1][0]:
                    for j in range(len(markerSetPoint[i][1])):
                        ID = int(markerSetPoint[i][1][j][1])
                        for k in range(len(markerID)):
                            if markerID[k][0] == ID:
                                self.match.append([markerSetPoint[i][1][j][0],markerID[k][1],markerID[k][2]])                 
            else:
                if framePos >= markerSetPoint[i][0]:
                    for j in range(len(markerSetPoint[i][1])):
                        ID = int(markerSetPoint[i][1][j][1])
                        for k in range(len(markerID)):
                            if markerID[k][0] == ID:
                                self.match.append([markerSetPoint[i][1][j][0],markerID[k][1],markerID[k][2]])
                    #match = [[KNE,402,278],[THI,410,262],[TIB,402, 318]]

    def get_list(self):
        return self.match
    
    def draw(self,frame,textSize = 0.5):
        self.textSize = textSize
        for i in range(len(self.match)):
            cv2.putText(frame, self.match[i][0], (self.match[i][1] - 10, self.match[i][2]- 5),cv2.FONT_HERSHEY_SIMPLEX, self.textSize, (255, 255, 255),2)
            cv2.circle(frame, (self.match[i][1] , self.match[i][2]), round(self.textSize *4),(255, 255, 255), -1)
        return frame
    
    
#track centroid from list of centoid coordinate base on distance in the previous frame and give each centroid an ID
#Init this class to start tracking use update() to feed list of coordinate in tracker system
#example of coordinate list: [(1244,486),(486,352)]
#get_list(): return list of all centroids id and coordinate in that image frame
    #example: [[0,1244,486],[1,486,352]]
#draw(): return an image of centroids ID
class CentroidTracker():
    def __init__(self, maxDisappeared=120):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()

        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, coor):
        # check to see if the list of input coordinate
        # is empty
        if len(coor) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # return early as there are no centroids or tracking info
            # to update
            #return self.objects

        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(coor), 2), dtype="int")

        # loop over the bounding box rectangles
        for i in range(len(coor)):
            # use the bounding box coordinates to derive the centroid
            cX = int(coor[i][0])
            cY = int(coor[i][1])
            inputCentroids[i] = (cX, cY)
            

        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])

        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()

            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]

            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        # return the set of trackable objects
        #return self.objects
    def get_list(self):
        self.markersID = []
        for (objectID, centroid) in self.objects.items():
            self.markersID.append([objectID,centroid[0],centroid[1]])
        return self.markersID
    
    def draw(self,frame,textSize = 0.5):
        self.frame = frame
        self.textSize = textSize
        for (objectID, centroid) in self.objects.items():
            text = "ID {}".format(objectID)
            cv2.putText(self.frame, text, (centroid[0] - 10, centroid[1] -15),cv2.FONT_HERSHEY_SIMPLEX, self.textSize, (0, 255, 0),2)
            cv2.circle(self.frame, (centroid[0], centroid[1]), round(self.textSize*4),(0, 255, 0), -1)
        return self.frame

In [4]:
#import lib for ML prediction
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
import pickle

In [5]:
#predict angle
class MyModel:
    #load model
    def __init__(self):
        filename = 'finalized_model.sav'
        self.loaded_model = pickle.load(open(filename, 'rb'))
    #predict angle
    def get_angle(self,coorlist):
        angle = self.loaded_model.predict(coorlist)
        return angle

In [16]:
#Window
class StartWindow:
    def __init__(self, master):
        self.master = master
        self.textFrame = Frame(self.master)
        self.textFrame.pack()
        self.buttonFrame = Frame(self.master)
        self.buttonFrame.pack()
        
        self.button1 = Button(self.buttonFrame, text = 'DEMO', command = self.demo)
        self.button1.pack(side = LEFT)
        self.button2 = Button(self.buttonFrame, text = 'browse video', command = self.browse)
        self.button2.pack(side = LEFT)
        self.welcomeText = Label(self.textFrame,text='Welcome to Gait Angle project\nby SunnyKungInwZaa007')
        self.welcomeText.pack(padx=200,pady=50)
        
        
    #open new window
    def new_window(self):
        self.newWindow = Toplevel(self.master)
        #if click demo do this
        if self.demoFlag:
            self.app = VideoDisplayWindow(self.newWindow,video_source="tester/sunny_walker3.mp4")
        #else is else
        else:
            self.app = VideoDisplayWindow(self.newWindow,video_source=str(self.master.filename))
    #browse the video file directory
    def browse(self):
        self.master.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("mp4 files","*.mp4"),("AVI files","*.avi")))
        if len(self.master.filename)>0:
            print (self.master.filename)
            self.demoFlag = 0
            self.new_window()
    #display demo video ข้าเองไงจะใครละ๕๕๕๕ อย่าแปลมันไม่ตรง
    def demo(self):
        self.demoFlag = 1
        self.new_window()
        
        
        
#Main Video display
class VideoDisplayWindow:
    def __init__(self, window,video_source):
        self.window = window
        self.video_source = video_source
        self.window.title("display")
        self.pause = 0
        self.trackerMaxFrameDisappeared = 20
        
        self.threshold_state = IntVar()
        self.markerID_state = IntVar()
        self.contour_state = IntVar()
        self.label_state = IntVar()
        
        self.marker_location = StringVar()
        self.marker_ID = StringVar()
        self.locationmenu = ["ASI", "PSI", "THI","KNE","TIB","ANK","TOE","HEE"]
        self.IDmenu = [0]
        
        #store markers ID and label in each frame for an entire video
        #[[0,[0,KNE,444,486]],[1,[0,None,456,458],[1,KNE,486,423]]]
        #[[frame 0,[ID 0,label,X 0,Y 0],...,[ID n,label,X n,Y n]],...,[frame n,[ID 0,label,X 0,Y 0],...,[ID n,label,X n,Y n]]
        self.markers = [] #<-- core
        
        #load model
        self.mymodel = MyModel()
        
        #take your time looking at this method
        #please go easy on it, it's big and long but not that hard
        self.display_layout()
        
        #initial some parametor
        self.video_framePosOld = 0
        self.threshValueOld = 0
        self.pair = []
        self.setPair = []
        # After it is called once, the videoUpdate method will be automatically called every delay milliseconds
        self.delay =15
        self.videoUpdate()
        self.window.mainloop()
    
    #Update display every 15 millisec. Seriously, this update the whole interface not just a video. I dont know why I name it like that.
    def videoUpdate(self):
        self.get_frame_ret, self.frame = self.vid.get_frame()
        if self.get_frame_ret:
            #Get video detail
            self.video_frameCount,self.video_framePos,self.time_duration = self.vid.get_vid_detail()
            self.video_framePos =round(self.video_framePos)
            
            #replay when the video end
            if(self.video_frameCount-1 == self.video_framePos):
                self.vid.set_vid_frame(0)
                
            #Get threshold value frome a slide bar
            self.threshValue = self.Slide_threshold.get()
            #Get binary image
            self.binary_image = BinaryImage(self.frame,threshValue=self.threshValue)
            self.binary_frame = self.binary_image.get_image()
            #Get a list of contours coordinate
            self.contour = Contour(self.binary_frame)
            self.contour_coor = self.contour.get_list()
            #get text size
            self.textSize = self.Slide_textSize.get()/10
            
            #detect if ID is writen
            if len(self.markers)>0:
                #run trough entire video markers once to crate list of marker for this frame to increse speed
                #most method will use the list
                for i in self.markers:
                    if i[0] == round(self.video_framePos):
                        self.currentFrameMarkers = i[1]
                        self.currentAngle = i[2]
                        break
                #print(self.video_framePos)
                #print(self.currentFrameMarkers)
            else: 
                self.currentFrameMarkers = []
                self.currentAngle = []
            
            #get fillter info and calculate output video  
            self.videoOutPut()
            #put output vidoe on canvas to display
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.out_frame))
            self.canvas.create_image(0, 0, image = self.photo, anchor = NW)
            self.label_duration.config(text = 'duration '+str(round(self.time_duration))+' millisec')
            
            #everything in this if is updated when change frame
            if abs(self.video_framePos - self.video_framePosOld)>0:
                self.updateIDlist()    
                
            self.video_framePosOld = self.video_framePos
            
            
            self.updateAddedPair()
            

            self.updateOutPut()
            self.updateAngle()
            
            #can seek while pause
            if(self.pause):
                self.vid.set_vid_frame(self.Slide_videoFrame.get())

            else:
                self.Slide_videoFrame.set(self.video_framePos)
            
                
        #This line keep destroy and build new window to refersh the window
        self.window.after(self.delay, self.videoUpdate)    
    
    def updateAngle(self):
        if len(self.currentAngle) >0:
            if self.currentAngle[1] is not "None":
                self.label_data10.config(text = "knee angle: "+str(self.currentAngle[1])+"degree")
            else:self.label_data10.config(text = "knee angle: ")
    
    def updateOutPut(self):
        #["ASI", "PSI", "THI","KNE","TIB","ANK","TOE","HEE"]
        ASI=1
        PSI=1
        THI=1
        KNE=1
        TIB=1
        ANK=1
        TOE=1
        HEE=1
        for i in self.currentFrameMarkers:
            if i[1] == 'ASI':
                ASI=0
                self.label_data1.config(text = "anterior hip: X: "+str(i[2])+" Y:"+str(i[3]))
            if i[1] == 'PSI':
                PSI=0
                self.label_data2.config(text = "posterior hip: X: "+str(i[2])+" Y:"+str(i[3]))
            if i[1] == 'THI':
                THI=0
                self.label_data3.config(text = "thigh: X: "+str(i[2])+" Y:"+str(i[3]))
            if i[1] == 'KNE':
                KNE=0
                self.label_data4.config(text = "knee: X: "+str(i[2])+" Y:"+str(i[3]))
            if i[1] == 'TIB':
                TIB=0
                self.label_data5.config(text = "tibia: X: "+str(i[2])+" Y:"+str(i[3]))
            if i[1] == 'ANK':
                ANK=0
                self.label_data6.config(text = "ankle: X: "+str(i[2])+" Y:"+str(i[3]))
            if i[1] == 'TOE':
                TOE=0
                self.label_data7.config(text = "toe: X: "+str(i[2])+" Y:"+str(i[3]))
            if i[1] == 'HEE':
                HEE=0
                self.label_data8.config(text = "heel: X: "+str(i[2])+" Y:"+str(i[3]))
        if ASI:
            self.label_data1.config(text = "anterior hip:")
        if PSI:
            self.label_data2.config(text = "posterior hip:")
        if THI:
            self.label_data3.config(text = "thigh:")
        if KNE:
            self.label_data4.config(text = "knee:")
        if TIB:
            self.label_data5.config(text = "tibia:")
        if ANK:
            self.label_data6.config(text = "ankle:")
        if TOE:
            self.label_data7.config(text = "toe:")
        if HEE:
            self.label_data8.config(text = "heel:")
            
    def writeID(self):
        self.ct = CentroidTracker(self.trackerMaxFrameDisappeared)
        temp = round(self.video_framePos)
        self.threshValue = self.Slide_threshold.get()
        
        markers_temp=[]
        for i in range(round(self.video_frameCount)):
            self.vid.set_vid_frame(i)
            self.get_frame_ret, self.frame = self.vid.get_frame()
            
            self.binary_image = BinaryImage(self.frame,threshValue=self.threshValue)
            self.binary_frame = self.binary_image.get_image()

            self.contour = Contour(self.binary_frame)
            self.contour_coor = self.contour.get_list()
            
            if len(self.contour_coor)>0:
                self.ct.update(self.contour_coor)
            
            a = []
            b = self.ct.get_list()
            #b = [[0, 462, 150], [1, 464, 126], [2, 0, 0], [3, 637, 323], [4, 630, 294], [5, 461, 159]]
            for j in range(len(b)):
                a.append([b[j][0],"None",b[j][1],b[j][2]])            
            markers_temp.append([i,a,["None","None","None"]])
            #a = [10, [[0, 'None', 462, 150], [1, 'None', 464, 126], [2, 'None', 0, 0], [3, 'None', 637, 323], [4, 'None', 630, 294], [5, 'None', 461, 159]]]

        self.vid.set_vid_frame(temp)     
        self.markers = markers_temp
        self.filter_3.select()
        self.filter_4.select()
        #print(self.markers)
        box.showinfo("notification", "writing finished")
    
        
    def drawMarkers(self,frame,ID = 1,label = 1):
        #[[0, 'None', 468, 123], [1, 'None', 466, 106], [2, 'None', 479, 90], [3, 'None', 562, 330], [4, 'None', 548, 303]\
        #, [5, 'None', 546, 277], [6, 'None', 469, 158], [7, 'None', 478, 88]]
        if len(self.currentFrameMarkers)>0:
            for j in self.currentFrameMarkers:
                if ID == 1:
                    text = "ID {}".format(j[0])
                    cv2.putText(frame, text, (j[2] - 10, j[3] -15),cv2.FONT_HERSHEY_SIMPLEX, self.textSize, (0, 255, 0),2)
                    cv2.circle(frame, (j[2], j[3]), round(self.textSize*4),(0, 255, 0), -1)
                if label == 1:
                    if j[1] is not "None":
                        text = j[1]
                        cv2.putText(frame, text, (j[2] - 10, j[3] -5),cv2.FONT_HERSHEY_SIMPLEX, self.textSize, (255, 255, 255),2)
                        cv2.circle(frame, (j[2] , j[3] ), round(self.textSize*4),(255, 255,255), -1)
            return frame
    def videoOutPut(self):
        self.out_frame = self.frame
        if(self.contour_state.get()):
            self.out_frame  = self.contour.draw(self.out_frame,textSize = self.textSize)
        if len(self.currentFrameMarkers) > 0:
            ID = 0
            if(self.markerID_state.get()):
                ID = 1
            label = 0
            if(self.label_state.get()):
                label = 1
            if ID or label:
                self.out_frame = self.drawMarkers(self.out_frame,ID,label)
        if(self.threshold_state.get()):
            self.out_frame = self.binary_frame
        
        if self.vid.overSize:
                    self.out_frame = cv2.resize(self.out_frame, (0,0), fx=self.vid.size_multiplyer, fy=self.vid.size_multiplyer)   
        
    #read self.pair and update the label    
    def updateAddedPair(self):
        self.added_pairs =  "added pairs:\n"
        for i in range(len(self.pair)):
            self.added_pairs = self.added_pairs+self.pair[i][0]+"= ID"+self.pair[i][1]+"\n"
        self.label_IDmatchDisplayFrame.config(text = self.added_pairs)
    
    #read marker ID and location from self.cirrentFrameMarkers and display them in drop down ID 
    def updateIDlist(self):
        #self.currentFrameMarkers: [[0, 'None', 678, 419], [1, 'None', 701, 887], [2, 'None', 675, 538]]
        # change options
        if len(self.currentFrameMarkers) > 0:
            self.ID = []
            for i in range(len(self.currentFrameMarkers)):
                self.ID.append(self.currentFrameMarkers[i][0])
            menu = self.option_marker_ID.children['menu']
            menu.delete(0,END)
            newmenu = self.ID
            for val in newmenu:
                menu.add_command(label=val,command=lambda v=self.marker_ID,l=val:v.set(l))
            self.marker_ID.set("ID")
        else:
            menu = self.option_marker_ID.children['menu']
            menu.delete(0,END)
            menu.add_command(label="None",command=lambda v=self.marker_ID,l="None":v.set(l))
        self.marker_ID.set("ID")
        
    #When push add button, this function put selected pair in self.pair     
    def AddPair(self):
        if self.marker_location.get() != 'location' and self.marker_ID.get() != 'None' and self.marker_ID.get() != 'ID' :
            #check if that location already exist
            for i in self.pair:
                if i[0] == self.marker_location.get():
                    self.pair.remove(i)
                    print(self.pair)
            #check if ID already exist
            for i in self.pair:
                if i[1] == self.marker_ID.get():
                    self.pair.remove(i)
                    print(self.pair)
            
            self.pair.append([self.marker_location.get(),self.marker_ID.get()])
            self.marker_location.set("location")
            self.marker_ID.set("ID")
            print(self.pair)
        else:
            box.showwarning('warning','Please select before proceed')
    
    
    def SetPair(self):
        if len(self.pair) > 0:
            isFrameExist = 0
            #Run trough memorized list of set pairs 
            for i in self.setPair:
                #self.setPair:[[25,[[KNE,0],[HEE,1]]],[40,[[KNE,1],[ANK,0]]],[80,[[KNE,1],[ANK,2],[HEE,0]]]]
                #check if set of pairs have already been set in this frame
                if round(self.video_framePos) == i[0]:
                    isFrameExist = 1
                    for j in self.pair:
                        #self.pair:[[KNE,0]]
                        for k in i[1]:
                            #i[1] = [[KNE,1],[HEE,0]]
                            #check if that pair already exist
                            if j[0] == k[0] or k[0] == 'DEL':
                                #remove it
                                i[1].remove(k)
                        for k in i[1]:
                            #check if that ID alreay been paired
                            if j[1] == k[1] or k[1] == 'DEL':
                                #remove it
                                i[1].remove(k)
                        #after delect every peieces of shitty pair from an existance, add the true god    
                        i[1].append(j) 
                #that frame set pair doesn't exist
            if isFrameExist == 0:
                self.setPair.append([round(self.video_framePos),self.pair])
            self.pair = []    
            self.setPair = sorted(self.setPair)
            print(self.setPair)
            self.updateMarkersLabel()
            
        else:
            box.showwarning('warning','Please add matched pair before proceed')     
        
    #Next, relabel ID in self.markers
    def updateMarkersLabel(self):
        for i in self.markers:
            #self.marker:[[25,[[0, 'None', 468, 123], [1, 'None', 466, 106], [2, 'None', 479, 90], [3, 'None', 562, 330], [4, 'None', 548, 303]\
            #, [5, 'None', 546, 277], [6, 'None', 469, 158], [7, 'None', 478, 88]]]]
            for j in range(len(self.setPair)):
                #self.setPair:[[25,[[KNE,0],[HEE,1]]],[40,[[KNE,1],[ANK,0]]],[80,[[KNE,1],[ANK,2],[HEE,0]]]]
                #check if j is the last element index in setPair lsit
                if j is not len(self.setPair)-1:
                    #check if frame in markers list is in srtPair frame interval
                    if i[0] >= self.setPair[j][0] and i[0]<self.setPair[j+1][0]:
                        #update marker label
                        for k in self.setPair[j][1]:
                            #self.setPair[j][1]:[[KNE,0],[HEE,1]]
                            for l in i[1]:
                                #i[1]:[[0, 'None', 468, 123], [1, 'None', 466, 106], [2, 'None', 479, 90], [3, 'None', 562, 330], [4, 'None', 548, 303]\
                                #, [5, 'None', 546, 277], [6, 'None', 469, 158], [7, 'None', 478, 88]]
                                #check for delete command
                                if k[0] == 'DEL':
                                    l[1] = 'None'
                                #check if k[0] is aleready exist in l[1]
                                elif k[0] == l[1]:
                                    #check if ID is the same
                                    #if it is the same, just let it be. if not change it to none
                                    if int(k[1]) != int(l[0]):
                                        l[1] = 'None'
                                else:
                                    #check if ID is the same
                                    #if it is the same, relabel it
                                    if int(k[1]) == int(l[0]):
                                        #relabel it
                                        l[1] = k[0]
                else:
                    if i[0] >= self.setPair[j][0]:
                        #update marker label
                        for k in self.setPair[j][1]:
                            #self.setPair[j][1]:[[KNE,0],[HEE,1]]
                            for l in i[1]:
                                #i[1]:[[0, 'None', 468, 123], [1, 'None', 466, 106], [2, 'None', 479, 90], [3, 'None', 562, 330], [4, 'None', 548, 303]\
                                #, [5, 'None', 546, 277], [6, 'None', 469, 158], [7, 'None', 478, 88]]
                                #check for delete command
                                if k[0] == 'DEL':
                                    l[1] = 'None'
                                #check if k[0] is aleready exist in l[1]
                                elif k[0] == l[1]:
                                    #check if ID is the same
                                    #if it is the same, just let it be. if not change it to none
                                    if int(k[1]) != int(l[0]):
                                        l[1] = 'None'
                                else:
                                    #check if ID is the same
                                    #if it is the same, relabel it
                                    if int(k[1]) == int(l[0]):
                                        #relabel it
                                        l[1] = k[0]
                                        
    def Clear(self): #clear
        self.pair = []
        print(self.pair)
    #put delete command in that frame
    def Reset(self):
        for i in self.setPair:
            #remove pair in that frame
            if round(self.video_framePos) == i[0]:
                self.setPair.remove(i)
        self.setPair.append([round(self.video_framePos),[['DEL','DEL']]])
        self.setPair = sorted(self.setPair)
        #print(self.setPair)
        self.updateMarkersLabel()

    def ResetAll(self):
        self.setPair = []
        self.setPair.append([0,[['DEL','DEL']]])
        print(self.setPair)
        self.updateMarkersLabel()
    def showPair(self):
        box.showinfo("show set", str(self.setPair))
    
    #calculate angle by rewrite over with new approximation
    def calangle(self):
        if len(self.markers)>0:
            for i in self.markers:
                hipangle_list = []
                kneeangle_list = []
                ankleangle_list = []
                for j in i[1]:
                    if j[1] == 'THI':
                        kneeangle_list.append(j[2]/self.vid.width_origin)
                        kneeangle_list.append(j[3]/self.vid.height_origin)
                    if j[1] == 'KNE':
                        kneeangle_list.append(j[2]/self.vid.width_origin)
                        kneeangle_list.append(j[3]/self.vid.height_origin)
                    if j[1] == 'TIB':
                        kneeangle_list.append(j[2]/self.vid.width_origin)
                        kneeangle_list.append(j[3]/self.vid.height_origin)
                i.remove(i[2])
                a = ["None","None","None"]
                if len(kneeangle_list) == 6:
                    print(kneeangle_list)
                    #self.mymodel.get_angle(kneeangle_list) = [angle]
                    a[1] = round(self.mymodel.get_angle([kneeangle_list])[0]/100-10,2)
                    print(self.mymodel.get_angle([kneeangle_list]))
                i.append(a)
                
                
    #save dialog for user to save
    def getSaveFileName(self):
        self.save_path =  filedialog.asksaveasfilename(initialdir = "/",title = "Select file",defaultextension=".csv"\
                                                       ,filetypes = (("csv files","*.csv"),("all files","*.*")))
        print( self.save_path)
    #saveExcel but normalrized
    def saveExcalN(self):
        self.saveExcal(train = 1)
    #crate dataframe frome self.markers and save data frame in excel
    def saveExcal(self,train = 0):
        
        if len(self.markers)>0:
            self.getSaveFileName()
            if len(self.save_path)>0:
                self.excelLabels = ['frame','milisec','ASIx','ASIy','PSIx','PSIy','THIx','THIy','KNEx','KNEy','TIBx','TIBy','ANKx','ANKy','TOEx','TOEy','HEEx'\
                                   ,'HEEy','hip angle','knee angle','ankle angle']
                self.excelData = []
                for i in self.markers:
                    tempTu = (i[0],'None')
                    ASI=PSI=THI=KNE=TIB=ANK=TOE=HEE=ASIx=ASIy=PSIx=PSIy=THIx=THIy\
                    =KNEx=KNEy=TIBx=TIBy=ANKx=ANKy=TOEx=TOEy=HEEx=HEEy\
                    =hip_Flag=hip_Val=knee_Flag=knee_Val=ankle_Flag=ankle_Val = 0
                    for j in i[1]:
                        if j[1] == 'ASI':
                            ASI=1
                            ASIx =j[2]
                            ASIy =j[3]
                            if train:
                                ASIx = int(ASIx)/self.vid.width_origin
                                ASIy = int(ASIy)/self.vid.height_origin
                        if j[1] == 'PSI':
                            PSI=1
                            PSIx =j[2]
                            PSIy =j[3]
                            if train:
                                PSIx = int(PSIx)/self.vid.width_origin
                                PSIy = int(PSIy)/self.vid.height_origin
                        if j[1] == 'THI':
                            THI=1
                            THIx =j[2]
                            THIy =j[3]
                            if train:
                                THIx = int(THIx)/self.vid.width_origin
                                THIy = int(THIy)/self.vid.height_origin
                        if j[1] == 'KNE':
                            KNE=1
                            KNEx=j[2]
                            KNEy=j[3]
                            if train:
                                KNEx = int(KNEx)/self.vid.width_origin
                                KNEy = int(KNEy)/self.vid.height_origin
                        if j[1] == 'TIB':
                            TIB=1
                            TIBx=j[2]
                            TIBy=j[3]
                            if train:
                                TIBx = int(TIBx)/self.vid.width_origin
                                TIBy = int(TIBy)/self.vid.height_origin
                        if j[1] == 'ANK':
                            ANK=1
                            ANKx=j[2]
                            ANKy=j[3]
                            if train:
                                ANKx = int(ANKx)/self.vid.width_origin
                                ANKy = int(ANKy)/self.vid.height_origin
                        if j[1] == 'TOE':
                            TOE=1
                            TOEx=j[2]
                            TOEy=j[3]
                            if train:
                                TOEx = int(TOEx)/self.vid.width_origin
                                TOEy = int(TOEy)/self.vid.height_origin
                        if j[1] == 'HEE':
                            HEE=1
                            HEEx=j[2]
                            HEEy=j[3]
                            if train:
                                HEEx = int(HEEx)/self.vid.width_origin
                                HEEy = int(HEEy)/self.vid.height_origin
 
                    if ASI:tempTu = tempTu + (ASIx,ASIy,)
                    else:tempTu = tempTu+('None','None',)
                    if PSI:tempTu = tempTu + (PSIx,PSIy,)
                    else:tempTu = tempTu+('None','None',)
                    if THI:tempTu = tempTu + (THIx,THIy,)
                    else:tempTu = tempTu+('None','None',)
                    if KNE:tempTu = tempTu + (KNEx,KNEy,)
                    else:tempTu = tempTu+('None','None',)
                    if TIB:tempTu = tempTu + (TIBx,TIBy,)
                    else:tempTu = tempTu+('None','None',)
                    if ANK:tempTu = tempTu + (ANKx,ANKy,)
                    else:tempTu = tempTu+('None','None',)
                    if TOE:tempTu = tempTu + (TOEx,TOEy,)
                    else:tempTu = tempTu+('None','None',)
                    if HEE:tempTu = tempTu + (HEEx,HEEy,)
                    else:tempTu = tempTu+('None','None',)
                    
                    if i[2][0] is not "None":
                        tempTu = tempTu +(i[2][0],)
                    else:tempTu = tempTu+('None',)
                    if i[2][1] is not "None":
                        tempTu = tempTu +(i[2][1],)
                    else:tempTu = tempTu+('None',)
                    if i[2][2] is not "None":
                        tempTu = tempTu +(i[2][2],)
                    else:tempTu = tempTu+('None',)
                    self.excelData.append(tempTu)
                #print(self.excelData)
                self.markersDataFrame = pd.DataFrame.from_records(self.excelData, columns=self.excelLabels)
                #print(self.markersDataFrame)

                export_csv = self.markersDataFrame.to_csv (self.save_path, index = None, header=True) 
            
        else:
            box.showwarning('warning','Please write markers before proceed')     
    def snapshot(self):
        # Get a frame from the video source
        ret, frame = self.vid.get_frame()
        if ret:
            cv2.imwrite("frame-" + time.strftime("%d-%m-%Y-%H-%M-%S") + ".jpg", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
   
    def setPlayPause(self):
        if self.pause:
            self.pause = 0
            
            print('play')
        else:
            self.pause = 1
            self.Slide_videoFrame.set(self.video_framePos+1)
            print('pause')
    def close_windows(self):
        self.window.destroy()
        
    def display_layout(self):
        #manage frame
        self.videoFrame = Frame(self.window )
        self.inputSettingFrame = Frame(self.window)
        self.videoFilterFrame = Frame(self.window)
        self.filterTuneFrame = Frame(self.window)
        self.IDmatchFrame = Frame(self.window)
        self.IDmatchToolFrame = Frame(self.window)
        self.IDmatchDisplayFrame = Frame(self.window)
        self.dataDisplayFrame = Frame(self.window)
        self.saveFrame = Frame(self.window)
        self.otherButtonFrame = Frame(self.window)
        #Diasplay all windowFrame
        self.inputSettingFrame.grid(row=0, column=0,sticky = NW)
        self.videoFilterFrame.grid(row=1, column=0,sticky = NW)
        self.filterTuneFrame.grid(row=2, column=0,sticky = NW)
        self.IDmatchFrame.grid(row=3, column=0,sticky = NW)
        self.IDmatchToolFrame.grid(row = 4,column = 0,sticky = NW)
        self.IDmatchDisplayFrame.grid(row=5, column=0,rowspan=9,sticky = NW)
        
        self.videoFrame.grid(row=0, column=1,sticky = N,rowspan=10)
        self.dataDisplayFrame.grid(row=0, column=2,sticky = NW,rowspan=3)
        self.saveFrame.grid(row=4,column = 2,sticky = NW)
        self.otherButtonFrame.grid(row=10, column=2,sticky = N)
        
        
        # open video source (by default this will try to open the computer webcam)
        self.vid = MyVideoCapture(self.video_source)
        
        #videoFrame label
        self.label_videoFrame = Label(self.videoFrame,text = "VIDEO",bg = 'blue',fg = 'white')
        self.label_videoFrame.pack(side = TOP,fill=X)
        # Create a canvas that can fit the above video source size
        self.canvas = Canvas(self.videoFrame, width = self.vid.width, height = self.vid.height)
        self.canvas.pack()
        #Time duration
        self.label_duration = Label(self.videoFrame)
        self.label_duration.pack()
        # Button that lets the user take a snapshot
        self.btn_snapshot=Button(self.videoFrame, text="Snapshot", width=50, command=self.snapshot)
        self.btn_snapshot.pack(anchor=CENTER, expand=True)
        #Button to pause and play
        self.btn_playpause=Button(self.videoFrame, text="Play/Pause", width=50, command=self.setPlayPause)
        self.btn_playpause.pack(anchor=CENTER, expand=True)
        #Video frame Seeker
        self.video_frameCount,self.video_framePos,__ = self.vid.get_vid_detail()
        self.Slide_videoFrame = Scale(self.videoFrame, from_=self.video_framePos, to=self.video_frameCount-1, length=600,\
                                      tickinterval=round(1/20*self.video_frameCount), orient=HORIZONTAL) 
        self.Slide_videoFrame.pack()
        
        #input setting frame label
        self.label_inputSettingFrame = Label(self.inputSettingFrame,text = "INPUT SETUP",bg = 'blue',fg = 'white',width = 40)
        self.label_inputSettingFrame.grid(row = 0,column = 0,columnspan = 4,sticky = NW)
        #Threshold adjust
        self.label_setThreshold = Label(self.inputSettingFrame,text = "\nthreshold:")
        self.label_setThreshold.grid(row = 1,column = 0,sticky = NW)
        self.Slide_threshold= Scale(self.inputSettingFrame, from_=255,to=95, length=200,tickinterval=160, orient=HORIZONTAL) 
        self.Slide_threshold.set(170)
        self.Slide_threshold.grid(row = 1,column = 1,sticky = NW)
        #write marker ID
        self.btn_setID=Button(self.inputSettingFrame, text="write trackable spots ID",command = self.writeID)
        self.btn_setID.grid(row = 2,column = 0,columnspan = 2,sticky = NW)
        
        
        #video filter frame label
        self.label_videoFilterFrame = Label(self.videoFilterFrame,text = "FILTER",bg = 'blue',fg = 'white',width = 40)
        self.label_videoFilterFrame.grid(row = 0,column = 0,columnspan = 4,sticky = NW)
        #Video filter check box
        #Show Binary Image
        self.filter_1 = Checkbutton(self.videoFilterFrame,text = 'Theshold',variable = self.threshold_state,onvalue = 1,offvalue = 0)
        self.filter_1.grid(row = 1,column = 0,sticky = NW)
        #Show Contours
        self.filter_2 = Checkbutton(self.videoFilterFrame,text = 'Contour',variable = self.contour_state,onvalue = 1,offvalue = 0)
        self.filter_2.grid(row = 1,column = 1,sticky = NW)
        #Show Marker ID
        self.filter_3 = Checkbutton(self.videoFilterFrame,text = 'Marker ID',variable = self.markerID_state,onvalue = 1,offvalue = 0)
        self.filter_3.grid(row = 1,column = 2,sticky = NW)
        #Show Marker Label
        self.filter_4 = Checkbutton(self.videoFilterFrame,text = 'Label',variable = self.label_state,onvalue = 1,offvalue = 0)
        self.filter_4.grid(row = 1,column = 3,sticky = NW)
        
        #Set text size
        self.Slide_textSize= Scale(self.filterTuneFrame, from_=1,to=20, length=200,tickinterval=19, orient=HORIZONTAL) 
        self.Slide_textSize.set(5)
        self.Slide_textSize.grid(row = 1,column = 0,sticky = NW)
        
        
        #videoFrame label
        self.label_IDmatchFrame = Label(self.IDmatchFrame,text = "MARKER MATCHING",bg = 'blue',fg = 'white',width = 40)
        self.label_IDmatchFrame.grid(row = 0,column = 0,columnspan = 7,sticky = NW)
        #IDmatch drop list
        self.marker_location.set("location")
        self.option_marker_location = OptionMenu(self.IDmatchFrame,self.marker_location, *self.locationmenu)
        self.option_marker_location.grid(row = 1,column = 0,sticky = NW)
        self.label_match = Label(self.IDmatchFrame,text = " = ")
        self.label_match.grid(row = 1,column = 1,sticky = NW)
        self.marker_ID.set("ID")
        self.option_marker_ID = OptionMenu(self.IDmatchFrame,self.marker_ID, *self.IDmenu)
        self.option_marker_ID.grid(row = 1,column = 2,sticky = NW)
        self.btn_addPair = Button(self.IDmatchFrame,text = "add",command = self.AddPair)
        self.btn_addPair.grid(row = 1,column = 3,sticky = NW)
        self.btn_ClearPair = Button(self.IDmatchFrame,text = "clear",command = self.Clear)
        self.btn_ClearPair.grid(row = 1,column = 4,sticky = NW)
        self.btn_setlPair = Button(self.IDmatchToolFrame,text = "set here",command = self.SetPair)
        self.btn_setlPair.grid(row = 0,column = 0,sticky = NW)
        self.btn_resetPair = Button(self.IDmatchToolFrame,text = "reset here",command = self.Reset)
        self.btn_resetPair.grid(row = 0,column = 1,sticky = NW)
        self.btn_resetAllPair = Button(self.IDmatchToolFrame,text = "reset all",command = self.ResetAll)
        self.btn_resetAllPair.grid(row = 0,column = 2,sticky = NW)
        self.btn_showPair = Button(self.IDmatchToolFrame,text = "show set",command = self.showPair)
        self.btn_showPair.grid(row = 0,column = 3,sticky = NW)
        #display ID added
        self.label_IDmatchDisplayFrame = Label(self.IDmatchDisplayFrame,text = "added pairs:\n")
        self.label_IDmatchDisplayFrame.grid(row = 0,column = 0,sticky = NW)
        
        #videoFrame label
        self.label_dataDisplayFrame = Label(self.dataDisplayFrame,text = "OUTPUT",bg = 'blue',fg = 'white',width = 30)
        self.label_dataDisplayFrame.grid(row = 0,column = 0)
        #Data Display
        self.label_data1 = Label(self.dataDisplayFrame,text = "anterior hip: ")
        self.label_data1.grid(row = 1,column = 0,sticky = NW)
        self.label_data2 = Label(self.dataDisplayFrame,text = "posterior hip: ")
        self.label_data2.grid(row = 2,column = 0,sticky = NW)
        self.label_data3 = Label(self.dataDisplayFrame,text = "thigh: ")
        self.label_data3.grid(row = 3,column = 0,sticky = NW)
        self.label_data4 = Label(self.dataDisplayFrame,text = "knee: ")
        self.label_data4.grid(row = 4,column = 0,sticky = NW)
        self.label_data5 = Label(self.dataDisplayFrame,text = "tibia: ")
        self.label_data5.grid(row = 5,column = 0,sticky = NW)
        self.label_data6 = Label(self.dataDisplayFrame,text = "ankle: ")
        self.label_data6.grid(row = 6,column = 0,sticky = NW)
        self.label_data7 = Label(self.dataDisplayFrame,text = "toe: ")
        self.label_data7.grid(row = 7,column = 0,sticky = NW)
        self.label_data8 = Label(self.dataDisplayFrame,text = "heel: ")
        self.label_data8.grid(row = 8,column = 0,sticky = NW)
        self.btn_calangle = Button(self.dataDisplayFrame,text = "calculate angle",command = self.calangle)
        self.btn_calangle.grid(row = 9,column = 0,sticky = NW)
        self.label_data9 = Label(self.dataDisplayFrame,text = "hip angle: ")
        self.label_data9.grid(row = 10,column = 0,sticky = NW)
        self.label_data10 = Label(self.dataDisplayFrame,text = "knee angle: ")
        self.label_data10.grid(row = 11,column = 0,sticky = NW)
        self.label_data11 = Label(self.dataDisplayFrame,text = "ankle angle: ")
        self.label_data11.grid(row = 12,column = 0,sticky = NW)
        
        #saveFrame
        self.label_save = Label(self.saveFrame,text = "save: ")
        self.label_save.pack(side = LEFT)
        self.btn_saveExcel = Button(self.saveFrame, text = 'save excel',command = self.saveExcal)
        self.btn_saveExcel.pack(side = LEFT)
        self.btn_saveExcelN = Button(self.saveFrame, text = 'save excel n',command = self.saveExcalN)
        self.btn_saveExcelN.pack(side = LEFT)
        self.btn_saveVideo = Button(self.saveFrame, text = 'save video')
        self.btn_saveVideo.pack(side = LEFT)
        
        
        #Button to quit
        self.label_log = Label(self.otherButtonFrame,width = 20)
        self.label_log.pack(side = LEFT)
        self.quitButton = Button(self.otherButtonFrame, text = 'Quit', width = 10, command = self.close_windows)
        self.quitButton.pack(side = RIGHT)
        
def main(): 
    root = Tk()
    app = StartWindow(root)
    root.mainloop()        

In [17]:
#run this cell to start the program
if __name__ == '__main__':
    main()

pause
play
pause
play
pause
[['KNE', '5']]
[]
[['KNE', '4']]
[['KNE', '4'], ['THI', '5']]
[['KNE', '4'], ['THI', '5'], ['TIB', '3']]
[[23, [['KNE', '4'], ['THI', '5'], ['TIB', '3']]]]
C:/Users/sunny.DESKTOP-QGFGEEK/Desktop/rrrrr.csv
C:/Users/sunny.DESKTOP-QGFGEEK/Desktop/GitClone/one_camera_knee_angle/WalkingVideo2/L4.mp4
C:/Users/sunny.DESKTOP-QGFGEEK/Desktop/GitClone/one_camera_knee_angle/WalkingVideo2/L4.mp4
pause
[['THI', '2']]
[['THI', '2'], ['KNE', '3']]
[['THI', '2'], ['KNE', '3'], ['TIB', '4']]
[[135, [['THI', '2'], ['KNE', '3'], ['TIB', '4']]]]
C:/Users/sunny.DESKTOP-QGFGEEK/Desktop/GitClone/one_camera_knee_angle/Knee_markers_excel/excelL4N.csv
C:/Users/sunny.DESKTOP-QGFGEEK/Desktop/GitClone/one_camera_knee_angle/WalkingVideo2/L5.mp4
pause
play
pause
[['THI', '3']]
[['THI', '3'], ['KNE', '2']]
[['THI', '3'], ['KNE', '2'], ['TIB', '6']]
[[121, [['THI', '3'], ['KNE', '2'], ['TIB', '6']]]]
C:/Users/sunny.DESKTOP-QGFGEEK/Desktop/GitClone/one_camera_knee_angle/Knee_markers_excel/ex